In [5]:
def unique_values(rows, col):
    """Find the unique values for a column in a dataset."""
    return set([row[col] for row in rows])

In [30]:
from collections import Counter
def class_counts(rows):
    """Counts the number of each type of example in a dataset."""
    return Counter([row[-1] for row in rows])


In [31]:
training_data = [
    ['Green', 3, 'Apple'],
    ['Yellow', 3, 'Apple'],
    ['Red', 1, 'Grape'],
    ['Red', 1, 'Grape'],
    ['Yellow', 3, 'Lemon'],
]
header = ["color", "diameter", "label"]

In [34]:
def gini(rows):
    # 0 for pure, 1 for impure
    
    """Calculate the Gini Impurity for a list of rows.

    There are a few different ways to do this, I thought this one was
    the most concise. See:
    https://en.wikipedia.org/wiki/Decision_tree_learning#Gini_impurity
    """
    counts = class_counts(rows)
    impurity = 1
    for label in counts:
        probability_of_label = counts[label] / float(len(rows))
        impurity -= probability_of_label**2 # 1 - sum(probability)^2
    return impurity

In [ ]:
class DecisionTree:
    def __init__(self, x, y, idxs=None, min_leaf=5):
        # x = shuffled row of data
        # y = shuffled row of response variable
        # index = index of training data used to train the tree. can be less than equal to training set size
        # min_leaf = passed by user or default 5
        self.x, self.y, self.idxs, self.min_leaf = x, y, idxs, min_leaf
        
        self.n = len(idxs) # number of rows
        self.c = x.shape[1] # number of columns
        
        self.val = np.mean(y[idxs]) # output value at each node. Avg of the response variable values
        
        self.score = float('inf') # some initial score as infinity.
        
        self.find_varsplit()
        
    # for each column, find best split
    def find_varsplit(self):
        for i in range(self.c): 
            self.find_better_split(i) 
    
    # for a particular column, find best split
    def find_better_split(self, col):
        # purpose : get the best row to split on.
        # process : go through every row and and try to minimize the standard deviation
        
        x = self.x.values[self.idxs,var_idx] # subset of x (specific rows and columns)
        y = self.y[self.idxs] # specific rows of y
        
        # goto each row of data and check for score
        # if 'i'th row is the split, what is the score
        # compare to find lowest score
        for i in range(1,self.n-1): 
            lhs = x<=x[i] # lhs - boolean array of 1 each time its x[i] >= x
            rhs = x>x[i] # rhs - boolean array of 1 each time x > x[i] < x
            
            if rhs.sum()==0: continue # if everything went to the left split
            lhs_std = y[lhs].std()
            rhs_std = y[rhs].std()
            curr_score = lhs_std*lhs.sum() + rhs_std*rhs.sum() # current_score = weigthed avg
            if curr_score < self.score: 
                self.var_idx, self.score, self.split = var_idx, curr_score, x[i]
    
    # how to represent a tree object
    def __repr__(self):
        # __repr__ is  special function that gets called when you try to "print" a tree object.
        s = f'n: {self.n}; val:{self.val}'
        # n = number of rows in the shuffled data for that row, or for the matter of fact, in the tree.
        # val = mean of the response variable value for that node.
        
        if not self.is_leaf:
            
            s += f'; score:{self.score}; split:{self.split}; var:{self.split_name}'
            # score = score at that point. score is the weighted std_dev of the response variable values
            # split = the row of data that gives the best split.
            # split_name = the variable on which the node is spliting. basically the variable name for that node.
            
        return s

    def split_name(self): 
        return self.x.columns[self.var_idx]
        # returns the column on which split has happened x.columns[var_idx]
        
    def split_col(self): 
        return self.x.values[self.idxs,self.var_idx]
        # returns a list of data value at x[idxs, var_idx]
        # eg. df_raw.values[[1,2,3], 2] = will return 3 values of column 2 at rows 1,2 and 3
    
    def is_leaf(self): 
        return self.score == float('inf') 
        # everything that has a split, has a score to split on.
        # this returns whether or not a node is a leaf, by comparing the score woth infinity.
    
    